In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras import layers, activations
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import mean_absolute_error

from tinymlgen import port

import numpy as np
from numpy import asarray

from utils.preprocessing import preprocessImage
from utils.grad_cam import get_img_array, make_gradcam_heatmap, save_and_display_gradcam
from utils.image_display import imageDisplay
from utils.transform_data import transformData

In [7]:
#Preprocessing
resize_image = True
#new_size = (480, 640)
new_size = (640, 480)
grey_scale = True
rgb_or_grey = 3
normalize_image = False
thresholding = False
erosion = False
kernel = np.ones((2, 2), np.uint8)
cut_image = False
turn_image = False

#Scaling
min_max = False
z_score = True

min_max_labels = False
z_score_labels = False

#Training
batch_size = 16
epochs = 150

if resize_image == True:
    #new_size = (300, 225)
    #new_size = (225, 300)
    if turn_image == True:
        #new_size = (160, 120)
        new_size = (120, 160)
        #new_size = (240, 320)
    else:
        new_size = (160, 120)
        #new_size = (320, 240)

if grey_scale == True:
    rgb_or_grey = 1

preprocessImage = preprocessImage()

In [8]:
imageFoldersValidation = "../Bilder/Manometer/Blindset final test"

#Load in Images and Clean DataFrame
validation_images, validation_labels = preprocessImage.load_images(imageFoldersValidation, True, grey_scale, resize_image, new_size, normalize_image, thresholding, erosion, kernel, cut_image, turn_image)

validation_images = validation_images.reshape(len(validation_labels), new_size[1], new_size[0], rgb_or_grey)
nRows,nCols,nDims = validation_images.shape[1:]
validation_images = validation_images.reshape(validation_images.shape[0], nRows, nCols, nDims)
#Scale Data
transformer = transformData()

#Parameters Training data
mean_value = 114.31817292906746
standard_deviation = 77.30169303183406

min_value = 0
max_value = 255

#Parameters Targetr Data
mean_value_labels = 5.035714285714286
standard_deviation_labels = 3.0150275548430043

min_labels = 0
max_labels = 10


if min_max == True:
    validation_images = transformer.scale_min_max(validation_images, min_value, max_value)

if z_score == True:
    validation_images = transformer.scale_z_score(validation_images, mean_value, standard_deviation)


if min_max_labels == True:
    validation_labels = transformer.scale_min_max(validation_labels, min_labels, max_labels)

if z_score_labels == True:
    validation_labels = transformer.scale_z_score(validation_labels, mean_value_labels, standard_deviation_labels)


imageFolderPath: ../Bilder/Manometer/Blindset final test


In [9]:
image_num = len(validation_labels)-1

path = "models/model3/model.tflite"

from sklearn.metrics import mean_squared_error

preds = []

interpreter = tf.lite.Interpreter(model_path=path)
interpreter.allocate_tensors()

for element in range(len(validation_labels)):
    
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    input_data = validation_images[element].reshape(1, 120, 160, 1)#.flatten()
    input_data = np.float32(input_data)
    interpreter.set_tensor(input_details[0]['index'], input_data)

    predictions=interpreter.invoke()

    output_data = interpreter.get_tensor(output_details[0]['index'])

    preds.append(output_data[0])

dict = {"True Value":"Predction"}
for A, B in zip(validation_labels, preds):
    dict[A] = B[0]

dict


{'True Value': 'Predction',
 0.3: 0.20461242,
 0.5: 0.7723175,
 0.8: 0.7723175,
 0.0: 0.0,
 1.3: 1.2793097,
 1.5: 1.5558641,
 1.8: 1.8550508,
 1.0: 0.93708295,
 10.0: 9.872805,
 2.3: 2.5088234,
 2.5: 2.6385996,
 2.8: 2.6351726,
 2.0: 1.9964563,
 3.3: 3.4578342,
 3.5: 3.889193,
 3.8: 4.178481,
 3.0: 3.0984712,
 4.3: 4.652845,
 4.5: 4.8138943,
 4.8: 5.0273514,
 4.0: 4.3639083,
 5.3: 5.6443396,
 5.5: 5.9903326,
 5.8: 6.204909,
 5.0: 5.3425717,
 6.3: 6.6990905,
 6.5: 6.9512444,
 6.8: 7.299215,
 6.0: 6.3782773,
 7.3: 7.600967,
 7.5: 7.820777,
 7.8: 7.8140426,
 7.0: 7.475898,
 8.3: 8.466311,
 8.5: 8.501323,
 8.8: 8.566385,
 8.0: 8.030275,
 9.3: 9.326473,
 9.5: 9.811596,
 9.8: 9.938322,
 9.0: 9.024612}

In [10]:
errors = mean_absolute_error(validation_labels, preds)
errors

0.21631877051620949